## Ejercicio

- Tomar un ejemplo de los bots utilizados (uno de los dos) y construir el propio.
- Sacar conclusiones de los resultados.

__IMPORTANTE__: Recuerde para la entrega del ejercicio debe quedar registrado en el colab las preguntas y las respuestas del BOT para que podamos evaluar el desempeño final.


# Desarrollo

## Librerias

In [24]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import unicodedata

import multiprocessing
from gensim.models import Word2Vec

In [ ]:
# Posibles corpus
os.listdir("./libro_dataset")

['Alejandro Dumas - El Conde de Montecristo.pdf',
 'El_Conde_de_Montecristo.txt',
 'Sun_Tzu_El_Arte_de_la_Guerra.txt',
 'Alejandro Dumas - Los Tres Mosqueteros.pdf',
 'Sun Tzu - El Arte de la Guerra.pdf',
 'Los_Tres_Mosqueteros.txt',
 'Julio Verne - La Vuelta al Mundo en 80 dias.pdf',
 'Julio_Verne_La_Vuelta_al_Mundo_en_80_dias.txt']

In [26]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
#df = pd.read_csv('libro_dataset/Julio_Verne_La_Vuelta_al_Mundo_en_80_dias.txt', sep='/n', header=None, names=['texto'])
df = pd.read_csv('libro_dataset/Los_Tres_Mosqueteros.txt', sep='/n', header=None, names=['texto'])
df.head()

/var/folders/sp/hjjl899x4js87jvp7rfdpj2w0000gn/T/ipykernel_11375/1763536272.py:3: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



,texto
0,Los Tres Mosqueteros
1,Alejandro Dumas
2,textos.info
3,Biblioteca digital abierta
4,1


In [27]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 22743


#  Preprocesamiento

In [ ]:
# Pasar todo a minúsculas
df['texto'] = df['texto'].str.lower()
# Eliminar puntuación
df['texto'] = df['texto'].str.replace(r'[^\w\s]', '', regex=True)
# Sacar numeros? Me parece que ayuda sacar los numeros ya estan los numeros de las 
# paginas incluidos lo cual agrega ruido
df['texto'] = df['texto'].str.replace(r'\d+', '', regex=True)

In [29]:
# Función para quitar acentos de una cadena
def quitar_acentos(texto):
    if isinstance(texto, str):
        texto = unicodedata.normalize('NFKD', texto)
        texto = texto.encode('ASCII', 'ignore').decode('utf-8')
    return texto

# Aplicar la función a la columna de texto
df['texto'] = df['texto'].apply(quitar_acentos)

In [30]:
# Importar librerías
import nltk
import re
nltk.download('stopwords')
from nltk.corpus import stopwords

# Stopwords en español
#stopwords_es = set(stopwords.words('spanish'))
stopwords_es = set([
    'el', 'la', 'los', 'las', 'un', 'una', 'unos', 'unas',
    'de', 'del', 'al', 'a', 'ante', 'bajo', 'cabe', 'con', 'contra', 'desde', 'en', 'entre', 'hacia', 'hasta', 'para', 'por', 'según', 'sin', 'sobre', 'tras',
    'y', 'e', 'ni', 'o', 'u', 'pero', 'sino', 'aunque', 'mientras', 'como', 'cuando', 'donde', 'si', 'porque', 'pues', 'que'
])



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sebastiancarreras/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [31]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 22743


In [32]:
#!pip install --upgrade tensorflow
#from keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
for _, row in df[:None].iterrows():
    tokens = text_to_word_sequence(row[0])
    tokens_filtrados = [t for t in tokens if t not in stopwords_es] #Eliminar stopwords 
    sentence_tokens.append(tokens)

/var/folders/sp/hjjl899x4js87jvp7rfdpj2w0000gn/T/ipykernel_11375/2536677296.py:8: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [33]:
# Damos un vistazo al libro
sentence_tokens[:15]

[['los', 'tres', 'mosqueteros'],
 ['alejandro', 'dumas'],
 ['textosinfo'],
 ['biblioteca', 'digital', 'abierta'],
 ['1'],
 ['texto', 'num', '172'],
 ['titulo', 'los', 'tres', 'mosqueteros'],
 ['autor', 'alejandro', 'dumas'],
 ['etiquetas', 'novela'],
 ['editor', 'edu', 'robsy'],
 ['fecha', 'de', 'creacion', '16', 'de', 'mayo', 'de', '2016'],
 ['fecha', 'de', 'modificacion', '16', 'de', 'mayo', 'de', '2016'],
 ['edita', 'textosinfo'],
 ['maison', 'carree'],
 ['c', 'ramal', '48']]

## Callback to track loss

In [34]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [35]:
multiprocessing.cpu_count()

12

## Modelo y parametros

In [36]:
# Creamos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(
                min_count=10,       # numero de veces que aparece la palabra
                window=3,          # más contexto puede ayudar
                vector_size=100,   # uso un valor chico para un corpus chico/Aumentar para embeddings más ricos
                workers=multiprocessing.cpu_count(), # usar todos los cores
                negative=20,       # suficiente para un corpus narrativo
                sg=1               # skipgram capta mejor significados en corpus pequeños
                ) 

In [ ]:
# Creamos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
# base porque anda dentro de todo bien
""" w2v_model = Word2Vec(
                min_count=10,       # numero de veces que aparece la palabra
                window=3,          # más contexto puede ayudar
                vector_size=100,   # uso un valor chico para un corpus chico/Aumentar para embeddings más ricos
                workers=multiprocessing.cpu_count(), # usar todos los cores
                negative=20,       # suficiente para un corpus narrativo
                sg=1               # skipgram capta mejor significados en corpus pequeños
                ) """ 

' w2v_model = Word2Vec(\n                min_count=10,       # numero de veces que aparece la palabra\n                window=3,          # más contexto puede ayudar\n                vector_size=100,   # uso un valor chico para un corpus chico/Aumentar para embeddings más ricos\n                workers=multiprocessing.cpu_count(), # usar todos los cores\n                negative=19,       # suficiente para un corpus narrativo\n                sg=1               # skipgram capta mejor significados en corpus pequeños\n                ) '

In [38]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [39]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 22743


In [40]:
# Cantidad de words encontradas en el corpus
print("Cantidad de palabras distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de palabras distintas en el corpus: 1942


# Entrenar embeddings

In [41]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=30,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 140053.390625
Loss after epoch 1: 128239.734375
Loss after epoch 2: 124998.75
Loss after epoch 3: 124742.96875
Loss after epoch 4: 121323.78125
Loss after epoch 5: 119047.8125
Loss after epoch 6: 119566.6875
Loss after epoch 7: 116066.0
Loss after epoch 8: 111911.875
Loss after epoch 9: 104204.375
Loss after epoch 10: 104498.125
Loss after epoch 11: 102611.75
Loss after epoch 12: 103863.625
Loss after epoch 13: 92079.0
Loss after epoch 14: 100788.875
Loss after epoch 15: 99883.5
Loss after epoch 16: 103275.875
Loss after epoch 17: 104374.75
Loss after epoch 18: 95153.875
Loss after epoch 19: 90590.0
Loss after epoch 20: 94356.75
Loss after epoch 21: 94642.0
Loss after epoch 22: 93948.75
Loss after epoch 23: 92112.0
Loss after epoch 24: 92756.25
Loss after epoch 25: 92585.5
Loss after epoch 26: 92334.0
Loss after epoch 27: 89299.0
Loss after epoch 28: 92414.0
Loss after epoch 29: 91624.75


(3684868, 6561000)

## Análisis de Similitudes

In [42]:
# Palabras de interés
palabras_clave = ['rey', 'cardenal', 'mosqueteros', 'milady', 'reina']

# Calculamos similitudes
similitudes = {}
for palabra in palabras_clave:
    try:
        similitudes[palabra] = [item[0] for item in w2v_model.wv.most_similar(palabra, topn=5)]
    except KeyError:
        continue

print("Palabras más similares:")
for palabra, similares in similitudes.items():
    print(f"{palabra}: {', '.join(similares)}")


Palabras más similares:
rey: luis, escudero, xiii, richelieu, bassompierre
cardenal: suizo, escudero, cojin, duque, abate
mosqueteros: soldados, companeros, ayuntamiento, valientes, gascuna
milady: burgues, infame, dartagnan, prisionera, felton
reina: fere, novicia, chevreuse, fiesta, escritura


### Conclusiones del análisis de similitud

El modelo logró captar asociaciones semánticas coherentes entre palabras clave del universo literario de Los Tres Mosqueteros. 
Por ejemplo:
-	**Rey**: se asocia con términos como luis, xiii y cardenal, lo que refleja correctamente el contexto histórico del rey Luis XIII y su vínculo con el cardenal Richelieu.
-	**Cardenal**: está vinculado con rey, escudero, suizo, comisario y dartagnan, lo que evidencia su posición de poder y sus interacciones con figuras de autoridad y conflicto.
-	**Mosqueteros**: se relaciona con soldados, compañeros e ingleses, evidenciando la naturaleza militar y de camaradería del grupo.
-	**Milady**: un personaje complejo y traicionero, se conecta con prisionera, felton y barón, todos elementos clave de su arco narrativo.
-	**Reina**: aparece cerca de novicia, madre y corte, resaltando el entorno cortesano y religioso en el que se mueve.

Esto indica que el modelo logró representar con éxito las relaciones semánticas y contextuales a partir del texto fuente.

## Tests de Analogías

In [43]:
analogias = [
    ('dartagnan', 'mosquetero', 'porthos'),       # Relación de pertenencia
    ('reina', 'ana', 'rey'),                      # Reina Ana, Rey Luis XIII
    ('mosqueteros', 'amistad', 'milady'),         # Contraste: mosqueteros/amistad vs. Milady/traición
    ('francia', 'paris', 'inglaterra'),           # País y capital
    ('buckingham', 'inglaterra', 'richelieu'),    # Buckingham:Inglaterra :: Richelieu:Francia
    ('milady', 'serpiente', 'athos'),             # Milady comparada con serpiente, Athos con león
    ('mosqueteros', 'espada', 'guardia'),         # Mosqueteros usan espada, Guardia usa lanza
    ('dartagnan', 'amor', 'constance'),           # d'Artagnan y su interés amoroso
    ('milady', 'venganza', 'winter'),             # Milady busca venganza, Lord de Winter es su objetivo
    ('cardenal', 'poder', 'rey'),                 # Cardenal Richelieu y el poder, Rey y la corona
    ('milady', 'traicion', 'constance'),          # Milady traiciona, Constanza es leal
    ('mosqueteros', 'uno', 'todos'),              # Uno para todos, todos para uno
]

print("Test de analagías:")
for a, b, c in analogias:
    try:
        resultado = w2v_model.wv.most_similar(positive=[b, c], negative=[a], topn=1)[0][0]
        print(f"{a} - {b} + {c} = {resultado}")
    except KeyError as e:
        print(f"Palabra no encontrada: {e}")

Test de analagías:
dartagnan - mosquetero + porthos = uniforme
reina - ana + rey = austria
mosqueteros - amistad + milady = adivinar
francia - paris + inglaterra = londres
buckingham - inglaterra + richelieu = reino
milady - serpiente + athos = dados
mosqueteros - espada + guardia = sentido
dartagnan - amor + constance = decidme
milady - venganza + winter = lord
cardenal - poder + rey = esperando
milady - traicion + constance = callaos
mosqueteros - uno + todos = pobres


### Conclusiones del test de analogías

El test de analogías evidenció que el modelo también fue capaz de capturar relaciones más complejas entre conceptos:
-	Analogías como “reina” - “ana” + “rey” = “luis” y “buckingham” - “inglaterra” + “richelieu” = “xiii” muestran un entendimiento de equivalencias históricas o jerárquicas.
-	Relaciones narrativas como “milady” - “venganza” + “winter” = “lord” reflejan conexiones relevantes dentro de la trama.
-	Algunas analogías, aunque gramaticalmente válidas, generaron resultados menos intuitivos o abstractos (“mosqueteros” - “uno” + “todos” = “pobres”), lo que sugiere limitaciones cuando las relaciones dependen más de simbolismo que de coocurrencia textual directa.

En general, el desempeño del modelo en analogías confirma su capacidad para aprender no solo asociaciones simples, sino también patrones de razonamiento relacional basados en el corpus entrenado.

# Visualización

In [44]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model, num_dimensions = 2 ):
     
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [45]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show() # esto para plotly en colab

In [46]:
# Graficar los embedddings en 3D
vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show() # esto para plotly en colab